<a href="https://colab.research.google.com/github/infernoman16/NLP_Task/blob/main/Chatbot_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

In [2]:
up=files.upload()

Saving intents.json to intents.json


In [3]:
import pandas as pd
import io

In [4]:
df=pd.read_json(io.BytesIO(up['intents.json']))

In [5]:
df.head()

,intents
0,"{'tag': 'greeting', 'patterns': ['Hi there', '..."
1,"{'tag': 'goodbye', 'patterns': ['Bye', 'See yo..."
2,"{'tag': 'thanks', 'patterns': ['Thanks', 'Than..."
3,"{'tag': 'noanswer', 'patterns': [], 'responses..."
4,"{'tag': 'options', 'patterns': ['How you could..."


In [6]:
import nltk 
from nltk.stem import WordNetLemmatizer
import json
import pickle 

In [11]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [7]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [8]:
words=[]
classes=[]
documents=[]
ignore_words=['?','!']
data_file=open('intents.json').read()

In [9]:
intents=json.loads(data_file)

In [12]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
    w=nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w,intent['tag']))
    if intent ['tag'] not in classes:
      classes.append(intent['tag'])

In [14]:
lemmatizer=WordNetLemmatizer()

In [17]:
words=[lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words=sorted(list(set(words)))
classes=sorted(list(set(classes)))
print(len(documents), "documents")
print(len(classes),"classes",classes)
print(len(words),"unique lemmatize words",words)


47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatize words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [18]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [19]:
training =[]
output_empty=[0]*len(classes)
for doc in documents:
  bag=[]
  pattern_words=doc[0]
  pattern_words=[lemmatizer.lemmatize(word.lower()) for word in pattern_words]
for w in words:
  bag.append(1) if w in pattern_words else bag.append(0)
  output_row=list(output_empty)
  output_row[classes.index(doc[1])]=1
  training.append([bag,output_row])
random.shuffle(training)
training =np.array(training)
train_x=list(training[:,0])
train_y=list(training[:,1])
print('training data created')

training data created


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  del sys.path[0]


In [20]:
model=Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [21]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [22]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)


Epoch 1/200
18/18 [==============================] - 1s 1ms/step - loss: 1.6788 - accuracy: 0.5595 
Epoch 2/200
18/18 [==============================] - 0s 1ms/step - loss: 0.0226 - accuracy: 1.0000
Epoch 3/200
18/18 [==============================] - 0s 1ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 4/200
18/18 [==============================] - 0s 1ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 5/200
18/18 [==============================] - 0s 1ms/step - loss: 2.1801e-04 - accuracy: 1.0000
Epoch 6/200
18/18 [==============================] - 0s 1ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 7/200
18/18 [==============================] - 0s 1ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 8/200
18/18 [==============================] - 0s 1ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 9/200
18/18 [==============================] - 0s 1ms/step - loss: 9.8056e-04 - accuracy: 1.0000
Epoch 10/200
18/18 [==============================] - 0s 1ms/step - loss: 0.0026 - accuracy: 1.0000


In [23]:
model.save('chatbot_model.h5', hist)

print("model created")

model created


In [26]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [27]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
  return sentence_words


In [28]:
def bow(sentence, words, show_details=True):
  sentence_words = clean_up_sentence(sentence)
  bag = [0]*len(words) 
  for s in sentence_words:
    for i,w in enumerate(words):
      if w == s:
        bag[i] = 1
        if show_details:
          print ("found in bag: %s" % w)
  return(np.array(bag))


In [29]:
def predict_class(sentence, model):
  p = bow(sentence, words,show_details=False)
  res = model.predict(np.array([p]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
  return return_list


In [30]:
def getResponse(ints, intents_json):
  tag = ints[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if(i['tag']== tag):
      result = random.choice(i['responses'])
      break
  return result

def chatbot_response(text):
  ints = predict_class(text, model)
  res = getResponse(ints, intents)
  return res